In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.consequents/submit_results.csv /content/consequent_results.csv

In [0]:
import pandas as pd

prefix = '/content/'

test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2],
    'start':a_df[3],
    'end': a_df[4]
})
c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2],
    'start':c_df[3],
    'end': c_df[4]
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})

a_df.head(10)

In [0]:
pip install allennlp==1.0.0rc1 allennlp-models==1.0.0rc1

In [0]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.syntax.biaffine_dependency_parser
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

prediction = predictor.predict(
  sentence="When I ask Robertson if any of the tests offered by Theranos could have saved Holmes's uncle, he takes a long pause."
)
prediction

In [0]:
def extract_dependents(prediction):
 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for ke, val in v.items():
      if (ke=='root'):
        for key, value in val.items():
          if (key == 'children'):
            return extract_children(value) 

In [0]:
def extract_children(obj):
  arr = list()
  deps = list()
  spans = list()

  def extract(obj, arr): 
    if isinstance(obj, dict):
      for k, v in obj.items():
        if isinstance(v, (dict, list)):
          extract(v, arr)     
        if k=='word':
          arr.append(v)

    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
    
          for key, value in item.items():
            if key =='spans' and len(value) > 0:
              spans.append(value)
            if key =='children':
              deps.append(value)  
            #if key =='attributes' and 'VERB' in value:
            #  deps.append(item)
        
        extract(item, arr)
      
    return arr

  words = extract(obj, arr)
  return words, spans, deps

In [0]:
def stringnify(dependent):
  clauses = list()
 
  for d in dependent:
    c = separate_children(d) # list of d['word']s
    print(c)
    clauses.append(c)

  return clauses

In [0]:
def separate_children(obj):
  arr = list()

  def separate(obj, arr): 
    if isinstance(obj, dict):

      for k, v in obj.items():
        if k != 'children':
          if isinstance(v, (dict, list)):
              separate(v, arr)
          elif k == 'word':      
              arr.append(v)

    elif isinstance(obj, list):
        for item in obj:
            separate(item, arr)
      
    return arr

  results = separate(obj, arr)
  return results

In [0]:
words, spans, deps = extract_dependents(prediction)
print(words)
print(spans)
print(deps)

for dep in deps:
  clauses = stringnify(dep)
  print(clauses)


In [0]:
def analyze(id, df, r_df):
  atext = str()
  astart = -1
  aend = -1
  arule = df.loc[df.id.str.contains(id)]
  if len(arule) == 0:
    return id, 0, atext, astart, aend, 0

  # First check based on rules
  #print(id,len(arule.index))
  
  alab ='0' # assume antecedent false
  count= 0  # counter of arules
  for a in arule.index: # [a] = index
    count = count + 1
    amid = arule['id'][a]  
    alab = arule['labels'][a] # rule
    astart = arule['start'][a]
    aend = arule['end'][a]
    atext = arule['text'][a]
    text = atext
    #print(amid,'candidate ='+atext)
    if alab == 1:
      return amid, alab, atext, astart, aend, 100 # positive!

    # If rules gave no certain "yes"(==1 above)
    apred = r_df.loc[r_df.id.str.contains(amid)]
    if len(apred) == 0:
      print(amid, ' not predicted')
      continue
    for p in apred.index: # [p] = index
      amid = apred['id'][p]  
      alab = apred['labels'][p] # prediction
      apredrule = df.loc[df.id.str.contains(amid)]
      if len(apredrule) == 0:
        print(amid, ' not found ')
        continue
      for i in apredrule.index:    
        astart = apredrule['start'][i]
        aend = apredrule['end'][i]
        atext = apredrule['text'][i]
        text = atext
        #print(a,amid,alab,atext)
        if alab == 1:
          if len(arule.index) > count:
            return amid, alab, atext, astart, aend, 10*count # uncertain
          return amid, alab, atext, astart, aend, 75 # certain       
  return amid, alab, atext, astart, aend, 25 # unpredicted

In [0]:
import pandas as pd

prefix = '/content/'

from tqdm import tqdm
from time import sleep

submit_df = pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])
report_df = pd.DataFrame(columns=['sentenceID','clabel','cprob','consequent','cstart','cend','gold_consequent','alabel','aprob','antecedent','astart','aend','gold_antecedent'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]
  #print(id)

  astart=-1
  aend = -1
  cstart=-1
  cend = -1
  text= str()
  
  aid, alab, antecedent, astart, aend, aprob = analyze(id, a_df, ra_df)
  cid, clab, consequent, cstart, cend, cprob = analyze(id, c_df, rc_df)

  if len(antecedent) > 0:
    text = antecedent      
  if len(consequent) > 0 and not (consequent in text):
    text = text + ' | ' + consequent
  elif len(text) > 0:
    astart= cstart
    aend = cend 
    consequent = '{}'
    cstart=-1
    cend = -1 
  else:
    print(id,'no result')
    continue

  report_row = pd.Series( data={
    'sentenceID': id,
    'clabel': clab,
    'cprob': cprob,
    'consequent': consequent, # comment out gold_,
    'cstart': cstart,
    'cend': cend,
    #'gold_consequent': test_df['consequent'][ind],
    'alabel': alab,
    'aprob': aprob,
    'antecedent': antecedent, 
    'astart': astart,
    'aend': aend,
    #'gold_antecedent': test_df['antecedent'][ind]
  }, name = str(id) )

  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  }, name = str(id) )

  submit_df = submit_df.append(result_row, ignore_index=True)
  report_df = report_df.append(report_row, ignore_index=True)

  #print(id, text, astart, aend, cstart, cend)
  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid']) 
report_df.to_csv(prefix+'report.csv',index=False, header=['sentenceID','clabel','cprob','consequent','cstart','cend','gold_consequent','alabel','aprob','antecedent','astart','aend','gold_antecedent'])
report_df.head(100) 

In [0]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/report.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/